In [ ]:
import zipfile
import os

# Replace 'cifake-dataset.zip' with your actual file name
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('cifake_dataset')

# List the contents to verify
os.listdir('cifake_dataset')


['test', 'train']

In [ ]:
import tensorflow as tf
import os
import random
import numpy as np
import matplotlib.pyplot as plt

# Set base path to your dataset
base_dir = '/content/cifake_dataset'

# Paths for each class
train_fake_dir = os.path.join(base_dir, 'train', 'FAKE')
train_real_dir = os.path.join(base_dir, 'train', 'REAL')

# Get all image paths and shuffle
fake_images = [os.path.join(train_fake_dir, fname) for fname in os.listdir(train_fake_dir)]
real_images = [os.path.join(train_real_dir, fname) for fname in os.listdir(train_real_dir)]

random.shuffle(fake_images)
random.shuffle(real_images)

# Select only 50 from each
fake_images = fake_images[:50]
real_images = real_images[:50]

# Combine and create labels (0 = REAL, 1 = FAKE)
all_images = fake_images + real_images
labels = [1]*50 + [0]*50  # 1 = FAKE, 0 = REAL

# Create a TensorFlow dataset
def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  # Normalize
    return image, label

# Create tf.data.Dataset
path_ds = tf.data.Dataset.from_tensor_slices((all_images, labels))
image_ds = path_ds.map(load_and_preprocess)

# Shuffle and batch
BATCH_SIZE = 16
dataset = image_ds.shuffle(100).batch(BATCH_SIZE)


In [ ]:
fake_images = fake_images[:50]
real_images = real_images[:50]

all_images = fake_images + real_images
labels = [1]*50 + [0]*50


In [ ]:
from sklearn.model_selection import train_test_split

train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_images, labels, test_size=0.2, stratify=labels, random_state=42
)


In [ ]:
# Reuse your load and preprocess function
def load_and_preprocess(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, label

# Wrap training data
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess).shuffle(80).batch(16)

# Wrap testing data
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess).batch(16)


In [ ]:
from tensorflow.keras import layers, models

# Build the CNN model
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
history = cnn_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.4056 - loss: 1.5311 - val_accuracy: 0.5000 - val_loss: 0.6752
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.5215 - loss: 0.6812 - val_accuracy: 0.5500 - val_loss: 0.6480
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5503 - loss: 0.6678 - val_accuracy: 0.5000 - val_loss: 0.6416
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5760 - loss: 0.6569 - val_accuracy: 0.5000 - val_loss: 0.6869
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6342 - loss: 0.6714 - val_accuracy: 0.7500 - val_loss: 0.5786
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.7179 - loss: 0.5820 - val_accuracy: 0.7000 - val_loss: 0.5351
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7535 - loss: 0.5460 - val_accuracy: 0.7000 - val_loss: 0.4779
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6349 - loss: 0.5868 - val_accuracy: 0.7500 - val_loss: 0.4543
Epoch 9/10
5/

In [ ]:
import numpy as np

# Get ground truth and predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = cnn_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.round(preds).flatten())  # Round sigmoid output to 0 or 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy:  {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall:    {recall:.2f}")
print(f"F1 Score:  {f1:.2f}")


Accuracy:  0.85
Precision: 0.82
Recall:    0.90
F1 Score:  0.86


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

# Build your model on top of VGG16
vgg_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
vgg_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
history_vgg = vgg_model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 74s 15s/step - accuracy: 0.4990 - loss: 1.9054 - val_accuracy: 0.5000 - val_loss: 1.2807
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 79s 15s/step - accuracy: 0.6509 - loss: 1.1604 - val_accuracy: 0.5000 - val_loss: 1.1529
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 67s 15s/step - accuracy: 0.4977 - loss: 1.2418 - val_accuracy: 0.8000 - val_loss: 0.4483
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 69s 15s/step - accuracy: 0.6059 - loss: 0.6984 - val_accuracy: 0.8500 - val_loss: 0.4049
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 60s 13s/step - accuracy: 0.7594 - loss: 0.5097 - val_accuracy: 0.8000 - val_loss: 0.4549
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 91s 15s/step - accuracy: 0.7804 - loss: 0.5444 - val_accuracy: 0.8500 - val_loss: 0.4606
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 69s 15s/step - accuracy: 0.8358 - loss: 0.4548 - val_accuracy: 0.8500 - val_loss: 0.3768
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 71s 15s/step - accuracy: 0.7941 - loss

In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = vgg_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.round(preds).flatten())

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"VGG16 - Accuracy:  {accuracy:.2f}")
print(f"VGG16 - Precision: {precision:.2f}")
print(f"VGG16 - Recall:    {recall:.2f}")
print(f"VGG16 - F1 Score:  {f1:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
VGG16 - Accuracy:  0.85
VGG16 - Precision: 1.00
VGG16 - Recall:    0.70
VGG16 - F1 Score:  0.82


In [ ]:
from transformers import ViTFeatureExtractor
from PIL import Image
import numpy as np

# Load the ViT feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# Function to extract features
def extract_vit_features(dataset, feature_extractor):
    features = []
    labels = []

    for images, batch_labels in dataset:
        images_np = images.numpy()
        for img, label in zip(images_np, batch_labels.numpy()):
            # Convert to PIL image
            pil_img = Image.fromarray((img * 255).astype(np.uint8))
            # Extract features (returns tensor with shape (1, 3, 224, 224))
            inputs = feature_extractor(images=pil_img, return_tensors="np")
            features.append(inputs["pixel_values"][0])  # shape: (3, 224, 224)
            labels.append(label)

    return np.array(features), np.array(labels)

# Run it for both training and testing datasets
X_train, y_train = extract_vit_features(train_ds, feature_extractor)
X_test, y_test = extract_vit_features(test_ds, feature_extractor)

print(f"Train shape: {X_train.shape}, Labels: {y_train.shape}")
print(f"Test shape:  {X_test.shape}, Labels: {y_test.shape}")


Train shape: (80, 3, 224, 224), Labels: (80,)
Test shape:  (20, 3, 224, 224), Labels: (20,)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Build the classifier
vit_classifier = Sequential([
    Flatten(input_shape=(3, 224, 224)),  # Flatten the extracted ViT features
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: real (0) vs AI (1)
])

# Compile
vit_classifier.compile(optimizer=Adam(),
                       loss='binary_crossentropy',
                       metrics=['accuracy'])

# Train the model
vit_classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5969 - loss: 9.7400 - val_accuracy: 0.6500 - val_loss: 11.1384
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step - accuracy: 0.7437 - loss: 15.7808 - val_accuracy: 0.6000 - val_loss: 16.2882
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 392ms/step - accuracy: 0.7750 - loss: 13.5386 - val_accuracy: 0.6000 - val_loss: 13.0016
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 377ms/step - accuracy: 0.6906 - loss: 13.7313 - val_accuracy: 0.7000 - val_loss: 12.6662
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.8023 - loss: 5.4842 - val_accuracy: 0.7500 - val_loss: 10.9836
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 371ms/step - accuracy: 0.8547 - loss: 5.7204 - val_accuracy: 0.7000 - val_loss: 9.7369
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 401ms/step - accuracy: 0.9195 - loss: 2.2634 - val_accuracy: 0.5500 - val_loss: 12.4873
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 630ms/step - accuracy: 0.9438 - loss: 2.1713 - val_accuracy: 0.6500 - val

In [ ]:
# Predict
y_pred = np.round(vit_classifier.predict(X_test).flatten())

# Metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"ViT Accuracy:  {acc:.2f}")
print(f"ViT Precision: {prec:.2f}")
print(f"ViT Recall:    {rec:.2f}")
print(f"ViT F1 Score:  {f1:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
ViT Accuracy:  0.75
ViT Precision: 0.73
ViT Recall:    0.80
ViT F1 Score:  0.76


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(train_ds, validation_data=test_ds, epochs=10)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5733 - loss: 1.8588 - val_accuracy: 0.5500 - val_loss: 0.7442
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.4809 - loss: 1.5765 - val_accuracy: 0.5500 - val_loss: 0.6008
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5790 - loss: 0.7024 - val_accuracy: 0.6500 - val_loss: 0.6119
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6938 - loss: 0.6488 - val_accuracy: 0.7500 - val_loss: 0.4994
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.6752 - loss: 0.6017 - val_accuracy: 0.7500 - val_loss: 0.5010
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8160 - loss: 0.4939 - val_accuracy: 0.8000 - val_loss: 0.4753
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8167 - loss: 0.4313 - val_accuracy: 0.8000 - val_loss: 0.4355
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.7898 - loss: 0.3634 - val_accuracy: 0.7500 - val_loss: 0.4202
Epoch 9/10
5/5 ━━━━━━━━

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Load VGG16 without top layer
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable = False

# Build model
inputs = Input(shape=(224, 224, 3))
x = vgg_base(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)
vgg_model = Model(inputs, outputs)

vgg_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

vgg_model.fit(train_ds, validation_data=test_ds, epochs=10)


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 80s 16s/step - accuracy: 0.5012 - loss: 0.7182 - val_accuracy: 0.5000 - val_loss: 0.6700
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 68s 15s/step - accuracy: 0.4651 - loss: 0.7397 - val_accuracy: 0.7500 - val_loss: 0.6574
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 69s 15s/step - accuracy: 0.5566 - loss: 0.6991 - val_accuracy: 0.7500 - val_loss: 0.6491
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 70s 15s/step - accuracy: 0.4089 - loss: 0.7129 - val_accuracy: 0.7500 - val_loss: 0.6397
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 68s 15s/step - accuracy: 0.5950 - loss: 0.6728 - val_accuracy: 0.7500 - val_loss: 0.6326
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 83s 15s/step - accuracy: 0.4601 - loss: 0.7000 - val_accuracy: 0.7000 - val_loss: 0.6224
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 61s 13s/step - accuracy: 0.5590 - loss: 0.6676 - val_accuracy: 0.7000 - val_loss: 0.6133
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 92s 15s/step - accuracy: 0.5976 - loss: 0.6573 - val_accuracy: 0.8500 - val_loss: 0.6036


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Get predictions
y_pred_vgg = []
y_true_vgg = []

for images, labels in test_ds:
    preds = vgg_model.predict(images)
    y_pred_vgg.extend(np.round(preds).flatten())
    y_true_vgg.extend(labels.numpy())

# Calculate metrics
acc_vgg = accuracy_score(y_true_vgg, y_pred_vgg)
prec_vgg = precision_score(y_true_vgg, y_pred_vgg)
rec_vgg = recall_score(y_true_vgg, y_pred_vgg)
f1_vgg = f1_score(y_true_vgg, y_pred_vgg)

print(f"VGG16 Accuracy:  {acc_vgg:.2f}")
print(f"VGG16 Precision: {prec_vgg:.2f}")
print(f"VGG16 Recall:    {rec_vgg:.2f}")
print(f"VGG16 F1 Score:  {f1_vgg:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
VGG16 Accuracy:  0.80
VGG16 Precision: 0.88
VGG16 Recall:    0.70
VGG16 F1 Score:  0.78


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Get predictions from CNN model
y_pred_cnn = []
y_true_cnn = []

for images, labels in test_ds:
    preds = cnn_model.predict(images)
    y_pred_cnn.extend(np.round(preds).flatten())
    y_true_cnn.extend(labels.numpy())

# Calculate metrics
acc_cnn = accuracy_score(y_true_cnn, y_pred_cnn)
prec_cnn = precision_score(y_true_cnn, y_pred_cnn)
rec_cnn = recall_score(y_true_cnn, y_pred_cnn)
f1_cnn = f1_score(y_true_cnn, y_pred_cnn)

print(f"CNN Accuracy:  {acc_cnn:.2f}")
print(f"CNN Precision: {prec_cnn:.2f}")
print(f"CNN Recall:    {rec_cnn:.2f}")
print(f"CNN F1 Score:  {f1_cnn:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
CNN Accuracy:  0.75
CNN Precision: 0.78
CNN Recall:    0.70
CNN F1 Score:  0.74


In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

In [ ]:
# Function to preprocess and predict
def predict_ai_art(image):
    # Resize and normalize the image
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Predict using VGG16 model
    prediction = vgg_model.predict(image)[0][0]

    # Format result
    label = "AI-generated" if prediction >= 0.5 else "Real Art"
    confidence = prediction if prediction >= 0.5 else 1 - prediction
    return f"{label} ({confidence:.2%} confidence)"


In [3]:
from google.colab import files
uploaded = files.upload()  # Select best_ai_art_model.h5

from tensorflow.keras.models import load_model
vgg_model = load_model("best_ai_art_model.h5")


Saving best_ai_art_model.h5 to best_ai_art_model.h5


In [5]:
import gradio as gr
import numpy as np
from PIL import Image


# Prediction function
def predict_ai_art(image):
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    prediction = vgg_model.predict(image)[0][0]
    label = "AI-generated" if prediction >= 0.5 else "Real Art"
    confidence = prediction if prediction >= 0.5 else 1 - prediction
    return f"{label} ({confidence:.2%} confidence)"

# Gradio interface
interface = gr.Interface(
    fn=predict_ai_art,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🎨 AI Art Detector",
    description="Upload an artwork to check if it was created by an AI or a human, powered by VGG16."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1e116ea72488d32910.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
